<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab5/lab6_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 6 : BERT

@copyright:
    (c) 2023. iKnow Lab. Ajou Univ., All rights reserved.

M.S. Student: Wansik-Jo (jws5327@ajou.ac.kr)

# For assignment

- Python code의 주석 처리되어있는 부분을 구현하면 됩니다.
- MD 형식의 Cell의 [BLANK] 부분을 채우면 됩니다.
- MD 형식의 Cell의 [ANSWER] 부분 이후에 답을 작성하면 됩니다.
- 조교에게 퀴즈의 답과 함께 코드 실행 결과를 보여준 뒤, BB에 제출 후 가시면 됩니다.

---


## 목차

1. BERT
2. WordPiece Tokenizer
3. BERT Pretraining
    - Masked Language Model
    - Next Sentence Prediction
4. Embedding
    - Positional Embedding
    - Token Embedding
    - Segment Embedding
5. BERT Architecture
    - Multi-Head Attention
    - Feed Forward Network
    - Layer Normalization
    - Residual Connection
    - Encoder
6. Training

## 1. BERT (Introduction)

### BERT

- [BERT](https://arxiv.org/abs/1810.04805)는 2018년 10월에 Google에서 발표한 모델로, Bidirectional Encoder Representations from Transformers의 약자이다.

- 이름에서도 알 수 있듯이, BERT는 Transformer Bi-Directional Encoder를 사용하여 사전 훈련된 언어 모델이다.

- 즉 Pretaining 과정을 통해 얻은 사전 훈련된 모델을 Fine-tuning을 통해 다양한 NLP Task에 사용할 수 있다.

![BERT_pretraining](./figure/BERT_pretraining.png)

- 본 실습에서는, BERT의 구조를 이해하고, 직접 구현한다. 또한, BERT의 Pretraining 과정을 역시 직접 구현하는 것을 목표로 한다.


### Data

- 본 실습에서는, [Cornell Movie-Dialogs Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)를 사용한다.

    - 220,000 conversational exchanges between 10,292 pairs of movie characters


In [ ]:
!pip install transformers datasets tokenizers
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -qq cornell_movie_dialogs_corpus.zip
!rm cornell_movie_dialogs_corpus.zip
!mkdir datasets
!mv cornell\ movie-dialogs\ corpus/movie_conversations.txt ./datasets
!mv cornell\ movie-dialogs\ corpus/movie_lines.txt ./datasets

- 다음과 같은 preprocess 과정을 거친다.

1. Data corpus는 `movie_conversations.txt` 파일과 `movie_lines.txt` 파일로 나뉘어져 있다.

2. `movie_lines.txt` 파일의 text를 특정 delimiter (‘+++ $ +++’)를 사용하여 line의 ID, character ID, movie ID, 그리고 dialogue text로 나누어 line_dic이라는 dictionary에 저장한다.

3. `movie_conversations.txt` 파일의 text를 특정 delimiter (‘+++ $ +++’)를 사용하여 character ID, character ID, movie ID, 그리고 dialogue text로 나누어 conv_dic이라는 dictionary에 저장한다.

4. 마지막으로, input sequence의 최대 길이를 64로 제한한다. 이를 위해 text를 split하고, 처음 64개의 token만을 사용한다.


In [ ]:
from pathlib import Path

MAX_LEN = 64

### loading all data into memory
corpus_movie_conv = './datasets/movie_conversations.txt'
corpus_movie_lines = './datasets/movie_lines.txt'
with open(corpus_movie_conv, 'r', encoding='iso-8859-1') as c:
    conv = c.readlines()
with open(corpus_movie_lines, 'r', encoding='iso-8859-1') as l:
    lines = l.readlines()

### splitting text using special lines
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

### generate question answer pairs
pairs = []
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []

        if i == len(ids) - 1:
            break

        first = lines_dic[ids[i]].strip()
        second = lines_dic[ids[i+1]].strip()

        qa_pairs.append(' '.join(first.split()[:MAX_LEN]))
        qa_pairs.append(' '.join(second.split()[:MAX_LEN]))
        pairs.append(qa_pairs)

In [ ]:
print(len(pairs))
print(pairs[42])

221616
['She okay?', 'I hope so.']


## 2. WordPiece Tokenizer

- BERT는 WordPiece Tokenizer를 사용한다.

- WordPiece Tokenizer는 [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/abs/1508.07909) 논문에서 제안된 방법으로, 단어를 subword로 나누는 방법이다.

>  “I like surfboarding!” → [‘[CLS]’, ‘i’, ‘like’, ‘surf’, ‘##board’, ‘##ing’, ‘!’, ‘[SEP]’] → [1, 48, 250, 4033, 3588, 154, 5, 2]

- WordPiece Tokenizer는 각 pair마다 score를 계산한다.

$$ score = \text{frequency of the pair} \div \text{frequency of the first word} \times \text{frequency of the second word} $$

Huggingface의 설명을 보자

> By dividing the frequency of the pair by the product of the frequencies of each of its parts, the algorithm prioritizes the merging of pairs where the individual parts are less frequent in the vocabulary. For instance, it won’t necessarily merge ("un", "##able") even if that pair occurs very frequently in the vocabulary, because the two pairs "un" and "##able" will likely each appear in a lot of other words and have a high frequency. In contrast, a pair like ("hu", "##gging") will probably be merged faster (assuming the word “hugging” appears often in the vocabulary) since "hu" and "##gging" are likely to be less frequent individually.



- 본 실습에서는, 위에서 import한 training corpus를 위해 WordPiece Tokenizer를 직접 training한다.

1. transformer library의 `BertWordPieceTokenizer`를 import한다.

2. Conversation text를 여러개의 txt file로 나눈다. (N=10000)

3. Parameter들을 설정한다.
    - `clean_text` : Remove control character
    - `handle_chinese_chars` : Handle Chinese characters
    - `strip_accents` : Remove accents
    - `lowercase` : Lowercase

4. tokenizer.train 함수를 사용하여, training을 진행한다. 함수의 training parameter는 다음과 같다.
    - `files` : training에 사용할 txt file들의 list
    - `vocab_size` : tokenizer의 vocab size
    - `min_frequency` : tokenizer의 min frequency
    - `limit_alphabet` : tokenizer의 alphabet limit
    - `wordpieces_prefix` : tokenizer의 wordpieces prefix
    - `special_tokens` : tokenizer의 special tokens

5. tokenizer.save_model 함수를 사용하여, tokenizer를 저장한다.

In [ ]:
import os
from tqdm import tqdm
import torch
from tokenizers import BertWordPieceTokenizer
import transformers, datasets
from transformers import BertTokenizer

os.makedirs('./data', exist_ok=True)
text_data = []
file_count = 0

for sample in tqdm([x[0] for x in pairs]):
    text_data.append(sample)

    # Save at every 10k samples
    if len(text_data) == 10000:
        with open(f'./data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

paths = [str(x) for x in Path('./data').glob('**/*.txt')]

### training own tokenizer
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

tokenizer.train(
    files=paths,
    vocab_size=30_000,
    min_frequency=5,
    limit_alphabet=1000,
    wordpieces_prefix='##',
    special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']
    )

os.makedirs('./bert', exist_ok=True)
tokenizer.save_model('./bert', 'bert')
tokenizer = BertTokenizer.from_pretrained('./bert/bert-vocab.txt', local_files_only=True)

/home/iknow/anaconda3/envs/cpwan/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 221616/221616 [00:00<00:00, 1972885.23it/s]


/home/iknow/anaconda3/envs/cpwan/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1706: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [ ]:
target_sentence = "I was walking on the bridge. People passed my by slowly. And the weather was cloudy."

print(tokenizer.tokenize(target_sentence))
print(tokenizer.encode(target_sentence))
print(tokenizer.decode(tokenizer.encode(target_sentence)))

['i', 'was', 'walking', 'on', 'the', 'bridge', '.', 'people', 'passed', 'my', 'by', 'slowly', '.', 'and', 'the', 'weather', 'was', 'cloud', '##y', '.']
[1, 48, 215, 2643, 192, 150, 2783, 17, 464, 3259, 218, 454, 5583, 17, 179, 150, 3005, 215, 4682, 114, 17, 2]
[CLS] i was walking on the bridge. people passed my by slowly. and the weather was cloudy. [SEP]


BERT vocab의 special tokens은 다음과 같다.

- [PAD] : 0, Padding token, Training 과정에서 gradient를 계산하지 않는다.
- [UNK] : 1, Unknown token, Vocab에 없는 token, replacement용 token이다.
- [CLS] : 2, Classification token, 문장의 시작 token(SOS)이며, 전체 문장의 representation을 담고 있다.
- [SEP] : 3, Separation token, 문장의 끝 token(EOS)이며, 첫번쨰 문장과 두번째 문장을 구분하는 역할을 한다.
- [MASK] : 4, Mask token, Masked Language Model을 위해 사용되는 token이다.

In [ ]:
print(tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]', '[PAD]', '[MASK]', '[UNK]']))

[1, 2, 0, 3, 4]


### 3. BERT Pretraining

LM(Language Model)에서 살펴본 방법과 같이 BERT는 Next Word Prediction을 하지 않는다.

대신 BERT는 다음과 같은 두가지 방법을 사용한다.

1. Masked Language Model
2. Next Sentence Prediction

### 3.1 Masked Language Model

- Masked Language Model은 다음과 같은 방법으로 학습한다.

Input sequence의 15%의 token을 [MASK] token으로 바꾸고, training 과정에서 이를 예측하도록 한다.

해당 방법은 실제로 적용할 떄, 다음과 같은 문제점이 있었다.

Model이 [MASK] token이 input으로 들어올 때만 예측하도록 학습되기 때문에, 실제로 fine-tuning을 할 때와 같이 [MASK] token이 input으로 들어오지 않을 때는 예측을 하지 못한다.

따라서, 15%의 token 중에서
- 80%는 [MASK] token으로,
- 10%는 random token으로,
- 10%는 원래 token으로 유지하는 방법을 사용한다.


### 3.2 Next Sentence Prediction

또한, BERT model이 두 문장 사이의 관계를 파악할 수 있도록, NSP라는 task를 추가한다.

NSP는 두 문장이 이어지는지 아닌지를 예측하는 task이다.

NSP는 다음과 같은 방법으로 학습한다.

Input sequence의 50%는 첫번째 문장, 50%는 두번째 문장으로 구성한다.

이 때, 50%의 문장 중에서 50%는 두 문장이 이어지는 문장, 50%는 두 문장이 이어지지 않는 문장으로 구성한다.

In [ ]:
from torch.utils.data import Dataset, DataLoader
import itertools
import random

class BERTDataset(Dataset):
    def __init__(self, data_pair, tokenizer, seq_len=64):
        self.lines = data_pair
        self.tokenizer = tokenizer
        self.seq_len = seq_len

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, item):
        # Step 1: get random sentence pair, either negative or positive (saved as is_next_label)
        t1, t2, is_next_label = self.get_sent(item)

        # Step 2: replace random words in sentence with mask / random words
        t1_random, t1_label = self.random_word(t1)
        t2_random, t2_label = self.random_word(t2)

        # Step 3:
        # Adding CLS and SEP tokens to the start and end of sentences
        t1 = [self.tokenizer.vocab['[CLS]']] + t1_random + [self.tokenizer.vocab['[SEP]']]
        t2 = t2_random + [self.tokenizer.vocab['[SEP]']]
        # Adding PAD token for labels
        t1_label = [self.tokenizer.vocab['[PAD]']] + t1_label + [self.tokenizer.vocab['[PAD]']]
        t2_label = t2_label + [self.tokenizer.vocab['[PAD]']]

        # Step 4:
        # Combine sentence 1 and 2 as one input
        segment_label = ([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:self.seq_len]
        bert_input = (t1 + t2)[:self.seq_len]
        bert_label = (t1_label + t2_label)[:self.seq_len]
        # Adding PAD tokens to make the sentence same length as seq_len
        padding = [self.tokenizer.vocab['[PAD]'] for _ in range(self.seq_len - len(bert_input))]
        bert_input.extend(padding), bert_label.extend(padding), segment_label.extend(padding)

        output = {"bert_input": bert_input,
                  "bert_label": bert_label,
                  "segment_label": segment_label,
                  "is_next": is_next_label}

        return {key: torch.tensor(value) for key, value in output.items()}

    def random_word(self, sentence):
        #Randomly mask words in sentence
        tokens = sentence.split()
        output = []
        output_label = []

        # 15% of the tokens would be replaced
        for i, token in enumerate(tokens):
            prob = random.random()

            # We don't want to mask the special tokens(0,CLS, -1,SEP)
            token_id = self.tokenizer(token)['input_ids'][1:-1]

            if prob < 0.15:
                prob /= 0.15

                # 80% chance change token to mask token
                if prob < 0.8:
                    for i in range(len(token_id)):
                        output.append(self.tokenizer.vocab['[MASK]'])

                # 10% chance change token to random token
                elif prob < 0.9:
                    for i in range(len(token_id)):
                        output.append(random.randrange(len(self.tokenizer.vocab)))

                # 10% chance change token to current token
                else:
                    output.append(token_id)

                output_label.append(token_id)

            else:
                output.append(token_id)
                for i in range(len(token_id)):
                    output_label.append(0)

        # flattening
        output = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output]))
        output_label = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output_label]))
        assert len(output) == len(output_label)
        return output, output_label

    def get_corpus_line(self, item):
        '''return sentence pair'''
        return self.lines[item][0], self.lines[item][1]

    def get_random_line(self):
        '''return random single sentence'''
        return self.lines[random.randrange(len(self.lines))][1]

    def get_sent(self, index):
        '''return random sentence pair'''
        t1, t2 = self.get_corpus_line(index)

        # negative or positive pair, for next sentence prediction
        if random.random() > 0.5:
            return t1, t2, 1
        else:
            return t1, self.get_random_line(), 0


위 코드는 다음과 같은 방법으로 Torch BERT용 Dataset을 구성한다.

1. Positive 또는 Negative 둘 중 임의의 문장 쌍을 선택하고, 두 문장이 이어지는지 아닌지 여부를 나타내는 label `is_next`를 저장한다.

2. 위에서 설명한 확률에 따라, 두 문장의 무작위 token을 masking한다. 이 때, masking된 token의 label `mask_label`을 저장한다. 그 다음, Token ID로 변환한다.

3. 각 문장에 `[CLS]`와 `[SEP]` token을 추가한다.

4. 두 문장을 합쳐 하나의 sequence로 만든다. 이 때, `[SEP]` token을 사용하여 두 문장을 구분한다. 또한, `[PAD]` token을 사용하여 sequence의 길이를 `seq_len`으로 맞춘다.

5. `segment_label`을 생성한다. `segment_label`은 각 token이 어느 문장에 속하는지를 나타내는 label이다. `[CLS]`와 `[SEP]` token은 0, 첫번째 문장은 0, 두번째 문장은 1로 설정한다.

---

정의한 BERTDataset으로 train data를 생성하고, 이를 DataLoader를 통하여 batch로 나눈다.

In [ ]:
import random

train_data = BERTDataset(
   pairs, seq_len=MAX_LEN, tokenizer=tokenizer)
train_loader = DataLoader(
   train_data, batch_size=32, shuffle=True, pin_memory=True)
sample_data = next(iter(train_loader))
print(train_data[random.randrange(len(train_data))])

{'bert_input': tensor([   1,  162,   11,   58,   40, 1372,   17,    2,  769,   17,   48,   11,
         265,  249,  569,  269,   17,    2,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]), 'bert_label': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'segment_label': tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'is_next': tensor(1)}


## 4. Embedding

- BERT는 다음과 같은 3가지의 embedding을 사용한다.
    - Token Embedding
    - Segment Embedding
    - Position Embedding

### 4.1 Positional Embedding

![BERT_positional_embedding](./figure/positional_embedding.png)

- BERT는 Transformer와 같이 Positional Embedding을 사용한다.

- Transformer와 다르게, BERT는 Positional Embedding을 학습한다.

- Positional Embedding은 다음과 같은 방법으로 학습한다.

    - Positional Embedding의 dimension은 `hidden_size`이다.

    - Positional Embedding의 index는 0부터 `max_len`까지이다.

    - Positional Embedding의 index를 `hidden_size`로 나눈 후, sin과 cos 함수를 적용한다.

    - Positional Embedding의 index가 짝수일 때는 sin 함수를, 홀수일 때는 cos 함수를 적용한다.

    - Positional Embedding의 index가 짝수일 때는 `hidden_size`의 앞 절반, 홀수일 때는 뒤 절반을 사용한다.

    $$ PE_{(pos, 2i)} = sin(pos / 10000^{2i / hidden\_size}) $$
    $$ PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / hidden\_size}) $$


In [ ]:
import math

class PositionalEmbedding(torch.nn.Module):
    def __init__(self, d_model, max_len=128):
        super().__init__()

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        for pos in range(max_len):
            # for each dimension of the each position
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))

        # include the batch size
        self.pe = pe.unsqueeze(0).to('cuda')

    def forward(self, x):
        return self.pe

- 위에서 직접 구현한 Positional Embedding과, 기본적인 embedding인 Token Embedding, 그리고 Segment Embedding을 합쳐 하나의 embedding matrix를 구성하는 `BertEmbedding`를 구현한다.

In [ ]:


class BERTEmbedding(torch.nn.Module):
    """
    BERT Embedding which is consisted with under features
        1. TokenEmbedding : normal embedding matrix
        2. PositionalEmbedding : adding positional information using sin, cos
        2. SegmentEmbedding : adding sentence segment info, (sent_A:1, sent_B:2)
        sum of all these features are output of BERTEmbedding
    """
    def __init__(self, vocab_size, embed_size, seq_len=64, dropout=0.1):
        """
        vocab_size: total vocab size
        embed_size: embedding size of token embedding
        dropout: dropout rate
        """
        super().__init__()
        self.embed_size = embed_size
        # (m, seq_len) --> (m, seq_len, embed_size)
        # padding_idx is not updated during training, remains as fixed pad (0)
        self.token = torch.nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.segment = torch.nn.Embedding(3, embed_size, padding_idx=0)
        self.position = PositionalEmbedding(d_model=embed_size, max_len=seq_len)
        self.dropout = torch.nn.Dropout(p=dropout)

    def forward(self, sequence, segment_label):
        x = self.token(sequence) + self.position(sequence) + self.segment(segment_label)
        x = self.dropout(x)

        return x

## 5. BERT Architecture

- BERT는 Transformer Encoder를 사용한다.

![BERT](./figure/BERT.png)

- Transformer Encoder는 다음과 같은 구조로 이루어져 있다.

    - Multi-Head Attention
    - Feed Forward Network
    - Layer Normalization
    - Residual Connection

### 5.1 Multi-Head Attention

Transformer에서 학습했던 것처럼, BERT는 Multi-Head Attention을 사용한다.

$$ MultiHead(Q, K, V) = Concat(head_1, ..., head_h)W^O $$
$$ head_i = Attention(QW_i^Q, KW_i^K, VW_i^V) $$
$$ Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V $$

In [ ]:
import torch.nn.functional as F

### attention layers
class MultiHeadedAttention(torch.nn.Module):

    def __init__(self, heads, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()

        assert d_model % heads == 0
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = torch.nn.Dropout(dropout)

        self.query = torch.nn.Linear(d_model, d_model)
        self.key = torch.nn.Linear(d_model, d_model)
        self.value = torch.nn.Linear(d_model, d_model)
        self.output_linear = torch.nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask):
        """
        query, key, value of shape: (batch_size, max_len, d_model)
        mask of shape: (batch_size, 1, 1, max_words)
        """
        # (batch_size, max_len, d_model)
        query = self.query(query)
        key = self.key(key)
        value = self.value(value)

        # (batch_size, max_len, d_model) --> (batch_size, max_len, h, d_k) --> (batch_size, h, max_len, d_k)
        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        key = key.view(key.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        value = value.view(value.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)

        # (batch_size, h, max_len, d_k) matmul (batch_size, h, d_k, max_len) --> (batch_size, h, max_len, max_len)
        scores = torch.matmul(query, key.permute(0, 1, 3, 2)) / math.sqrt(query.size(-1))

        # fill 0 mask with super small number so it wont affect the softmax weight
        # (batch_size, h, max_len, max_len)
        scores = scores.masked_fill(mask == 0, -1e9)

        # (batch_size, h, max_len, max_len)
        # softmax to put attention weight for all non-pad tokens
        # max_len X max_len matrix of attention
        weights = F.softmax(scores, dim=-1)
        weights = self.dropout(weights)

        # (batch_size, h, max_len, max_len) matmul (batch_size, h, max_len, d_k) --> (batch_size, h, max_len, d_k)
        context = torch.matmul(weights, value)

        # (batch_size, h, max_len, d_k) --> (batch_size, max_len, h, d_k) --> (batch_size, max_len, d_model)
        context = context.permute(0, 2, 1, 3).contiguous().view(context.shape[0], -1, self.heads * self.d_k)

        # (batch_size, max_len, d_model)
        return self.output_linear(context)

### 5.2 Feed Forward Network

Transformer의 Encoder와 같이, BERT는 Feed Forward Network를 사용한다.

두 개의 FC layer로 이루어져 있다. 두 FC layer 사이에는 [GeLU](https://arxiv.org/abs/1606.08415) activation function을 사용한다.

In [ ]:

class FeedForward(torch.nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, middle_dim=256, dropout=0.1):
        super(FeedForward, self).__init__()

        self.fc1 = torch.nn.Linear(d_model, middle_dim)
        self.fc2 = torch.nn.Linear(middle_dim, d_model)
        self.dropout = torch.nn.Dropout(dropout)
        self.activation = torch.nn.GELU()

    def forward(self, x):
        out = self.activation(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out

구현한 `MultiHeadAttention`과 `FeedForward`를 사용하여, 그리고 Torch module의 Layer Normalization을 사용하여 `EncoderLayer`를 구현한다.

In [ ]:
class EncoderLayer(torch.nn.Module):
    def __init__(
        self,
        d_model=768,
        heads=12,
        feed_forward_hidden=768 * 4,
        dropout=0.1
        ):
        super(EncoderLayer, self).__init__()
        self.layernorm = torch.nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadedAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model, middle_dim=feed_forward_hidden)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, embeddings, mask):
        # embeddings: (batch_size, max_len, d_model)
        # encoder mask: (batch_size, 1, 1, max_len)
        # result: (batch_size, max_len, d_model)
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        # residual layer
        interacted = self.layernorm(interacted + embeddings)
        # bottleneck
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded

최종적으로, BERT Model을 다음과 같이 구성할 수 있다.

- BERT
    - BertEmbedding
    - EncoderLayer * 12

- MLM , NSP

- BERTLM
    - BERT with MLM, NSP

In [ ]:
class BERT(torch.nn.Module):
    """
    BERT model : Bidirectional Encoder Representations from Transformers.
    """
    def __init__(self, vocab_size, d_model=768, n_layers=12, heads=12, dropout=0.1):
        """
        :vocab_size: vocab_size of total words
        :hidden: BERT model hidden size
        :n_layers: numbers of Transformer blocks(layers)
        :attn_heads: number of attention heads
        :dropout: dropout rate
        """
        super().__init__()
        self.d_model = d_model
        self.n_layers = n_layers
        self.heads = heads

        # paper noted they used 4 * hidden_size for ff_network_hidden_size
        self.feed_forward_hidden = 256

        # embedding for BERT, sum of positional, segment, token embeddings
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=d_model)

        # multi-layers transformer blocks, deep network
        self.encoder_blocks = torch.nn.ModuleList(
            [EncoderLayer(d_model, heads, 256, dropout) for _ in range(n_layers)])

    def forward(self, x, segment_info):
        # attention masking for padded token
        # (batch_size, 1, seq_len, seq_len)
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)

        # embedding the indexed sequence to sequence of vectors
        x = self.embedding(x, segment_info)

        # running over multiple transformer blocks
        for encoder in self.encoder_blocks:
            x = encoder.forward(x, mask)

        return x

class NextSentencePrediction(torch.nn.Module):
    """
    2-class classification model : is_next, is_not_next
    """

    def __init__(self, hidden):
        """
        :hidden: BERT model output size
        """
        super().__init__()
        self.linear = torch.nn.Linear(hidden, 2)
        self.softmax = torch.nn.LogSoftmax(dim=-1)

    def forward(self, x):
        # use only the first token which is the [CLS]
        return self.softmax(self.linear(x[:, 0]))

class MaskedLanguageModel(torch.nn.Module):
    """
    predicting origin token from masked input sequence
    n-class classification problem, n-class = vocab_size
    """

    def __init__(self, hidden, vocab_size):
        """
        :hidden: output size of BERT model
        :vocab_size: total vocab size
        """
        super().__init__()
        self.linear = torch.nn.Linear(hidden, vocab_size)
        self.softmax = torch.nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x))

class BERTLM(torch.nn.Module):
    """
    BERT Language Model
    Next Sentence Prediction Model + Masked Language Model
    """

    def __init__(self, bert: BERT, vocab_size):
        """
        :bert: BERT model which should be trained
        :vocab_size: total vocab size for masked_lm
        """
        super().__init__()
        self.bert = bert
        self.next_sentence = NextSentencePrediction(self.bert.d_model)
        self.mask_lm = MaskedLanguageModel(self.bert.d_model, vocab_size)

    def forward(self, x, segment_label):
        x = self.bert(x, segment_label)
        return self.next_sentence(x), self.mask_lm(x)

## 6. Training

구현한 BERTLM을 사용하여, BERT를 pretraining한다.

In [ ]:
import numpy as np
from torch.optim import Adam

class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''

    def __init__(self, optimizer, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        "Zero out the gradients by the inner optimizer"
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

In [ ]:
class BERTTrainer:
    def __init__(
        self,
        model,
        train_dataloader,
        test_dataloader=None,
        lr= 1e-4,
        weight_decay=0.01,
        betas=(0.9, 0.999),
        warmup_steps=10000,
        log_freq=10,
        device='cuda'
        ):

        self.device = device
        self.model = model
        self.train_data = train_dataloader
        self.test_data = test_dataloader

        # Setting the Adam optimizer with hyper-param
        self.optim = Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
        self.optim_schedule = ScheduledOptim(
            self.optim, self.model.bert.d_model, n_warmup_steps=warmup_steps
            )

        # Using Negative Log Likelihood Loss function for predicting the masked_token
        self.criterion = torch.nn.NLLLoss(ignore_index=0)
        self.log_freq = log_freq
        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))

    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def test(self, epoch):
        self.iteration(epoch, self.test_data, train=False)

    def iteration(self, epoch, data_loader, train=True):

        avg_loss = 0.0
        total_correct = 0
        total_element = 0

        mode = "train" if train else "test"

        # progress bar
        data_iter = tqdm(
            enumerate(data_loader),
            desc="EP_%s:%d" % (mode, epoch),
            total=len(data_loader),
            bar_format="{l_bar}{r_bar}"
        )

        for i, data in data_iter:

            # 0. batch_data will be sent into the device(GPU or cpu)
            data = {key: value.to(self.device) for key, value in data.items()}

            # 1. forward the next_sentence_prediction and masked_lm model
            next_sent_output, mask_lm_output = self.model.forward(data["bert_input"], data["segment_label"])

            # 2-1. NLL(negative log likelihood) loss of is_next classification result
            next_loss = self.criterion(next_sent_output, data["is_next"])

            # 2-2. NLLLoss of predicting masked token word
            # transpose to (m, vocab_size, seq_len) vs (m, seq_len)
            # criterion(mask_lm_output.view(-1, mask_lm_output.size(-1)), data["bert_label"].view(-1))
            mask_loss = self.criterion(mask_lm_output.transpose(1, 2), data["bert_label"])

            # 2-3. Adding next_loss and mask_loss : 3.4 Pre-training Procedure
            loss = next_loss + mask_loss

            # 3. backward and optimization only in train
            if train:
                self.optim_schedule.zero_grad()
                loss.backward()
                self.optim_schedule.step_and_update_lr()

            # next sentence prediction accuracy
            correct = next_sent_output.argmax(dim=-1).eq(data["is_next"]).sum().item()
            avg_loss += loss.item()
            total_correct += correct
            total_element += data["is_next"].nelement()

            post_fix = {
                "epoch": epoch,
                "iter": i,
                "avg_loss": avg_loss / (i + 1),
                "avg_acc": total_correct / total_element * 100,
                "loss": loss.item()
            }

            if i % self.log_freq == 0:
                data_iter.write(str(post_fix))
        print(
            f"EP{epoch}, {mode}: \
            avg_loss={avg_loss / len(data_iter)}, \
            total_acc={total_correct * 100.0 / total_element}"
        )

In [ ]:
'''test run'''

train_data = BERTDataset(
   pairs, seq_len=MAX_LEN, tokenizer=tokenizer)

train_loader = DataLoader(
   train_data, batch_size=128, shuffle=True, pin_memory=True)

bert_model = BERT(
  vocab_size=len(tokenizer.vocab),
  d_model=128,
  n_layers=2,
  heads=2,
  dropout=0.1
).to('cuda')

bert_lm = BERTLM(bert_model, len(tokenizer.vocab)).to('cuda')
bert_trainer = BERTTrainer(bert_lm, train_loader, device='cuda')
epochs = 5

for epoch in range(epochs):
  bert_trainer.train(epoch)

Total Parameters: 5703210


EP_train:0:   0%|| 1/1732 [00:01<38:22,  1.33s/it]

{'epoch': 0, 'iter': 0, 'avg_loss': 10.8682222366333, 'avg_acc': 52.34375, 'loss': 10.8682222366333}


EP_train:0:   1%|| 11/1732 [00:03<07:31,  3.81it/s]

{'epoch': 0, 'iter': 10, 'avg_loss': 10.883026903325861, 'avg_acc': 50.14204545454546, 'loss': 10.867362976074219}


...

{'epoch': 1, 'iter': 570, 'avg_loss': 6.034772616551762, 'avg_acc': 49.83855078809107, 'loss': 6.033595561981201}


EP_train:1:  34%|| 581/1732 [02:25<04:54,  3.91it/s]

{'epoch': 1, 'iter': 580, 'avg_loss': 6.035691063498466, 'avg_acc': 49.8171256454389, 'loss': 6.229894161224365}


EP_train:1:  34%|| 588/1732 [02:27<04:47,  3.98it/s]


KeyboardInterrupt: 